In [ ]:
regs = ["Ridge", "Lasso", "ElasticNet"]

for reg in regs:

    params = {"method": "batch", "lr": 0.1, "l": 0.1}
    mlflow.start_run(run_name=f"method-{params['method']}-lr-{params['lr']}-reg-{reg}", nested=True)
    
    print("="*5, reg, "="*5)

    # #######
    type_of_regression = str_to_class(reg)    #Ridge, Lasso, ElasticNet
    model = type_of_regression(**params)  
    model.fit(X_train, y_train)
    yhat = model.predict(X_test)
    mse  = model.mse(yhat, y_test)

    print("Test MSE: ", mse)
    mlflow.log_metric(key="test_mse", value=mse)

    signature = mlflow.models.infer_signature(X_train, model.predict(X_train))
    mlflow.sklearn.log_model(model, artifact_path='model', signature=signature)

    # #######

    mlflow.end_run()

<h1> Kyi Task 2 </h1>

In [ ]:
# (1) Define Regularization Method: Polynomial, Lasso, Ridge, Normal
regs = ["Ridge", "Lasso", "polynomial", "normal"]

# (2) Define Momentum
use_momentums = ["True", "False"]

# (3) Define method: sto, batch, mini
methods = ["stochastic", "batch", "minibatch"]

# (4) Define theta:
thetas = ["zeros", "xavier"]

# (5) Define learning rate:
lrs = [0.01, 0.001, 0.0001]

# Hyperparameter combination for expirement
hyperparameter_combinations = []

for use_momentum in use_momentums:
    for method in methods:
        for theta in thetas:
            for lr in lrs:
                params = {
                    "method": method,
                    "lr": lr,
                    "use_momentum": use_momentum == "True",
                    "momentum": 0.9,
                    "theta": theta,
                    "l": 0.1
                }

                hyperparameter_combinations.append(params)

# Loop through hyperparameter combinations
for reg in regs:
    for params in hyperparameter_combinations:
        print(params)
        with mlflow.start_run(run_name=f"method-{params['method']}-lr-{params['lr']}-theta-{params['theta']}-momentum-{params['momentum']}-reg-{reg}", nested=True):
            mlflow.log_params(params=params)
            print("=" * 5, reg, "=" * 5)
            print("method: ", params['method'])
            print("lr: ", params['lr'])
            print("use_momentum:", params['use_momentum'])
            print("theta :", params['theta'])

            if reg == 'normal':
                model = LinearRegression(regularizaiton = None)
                model.fit(X_train, y_train.values)
                # break
            elif reg == 'polynomial':
                poly_X_train = PolynomialFeatures(degree=2, include_bias=False).fit_transform(X_train)
                poly_X_test = PolynomialFeatures(degree=2, include_bias=False).fit_transform(X_test)
                
                model = LinearRegression(regularization=None)
                model.fit(poly_X_train, y_train.values)
            else:
                type_of_regression = str_to_class(reg)
                model = type_of_regression(**params)
                model.fit(X_train, y_train.values)
            
            # record average MSE and R2score for each hyper-parameter
            test_mse = model.avgMSE()  
            test_r2score = model.avgR2score()

            print("Test MSE: ", test_mse)
            mlflow.log_metric(key="test_mse", value=test_mse)

            print("Test R2 score: ", test_r2score)
            mlflow.log_metric(key="test_r2score", value=test_r2score)

            if reg == 'polynomial':
                signature = mlflow.models.infer_signature(X_train, model.predict(poly_X_train))
            else:
                signature = mlflow.models.infer_signature(X_train, model.predict(X_train))
                
            mlflow.sklearn.log_model(model, artifact_path='model', signature=signature)

    mlflow.end_run()

In [ ]:
model = Normal_LogisticRegression(k, X_train.shape[1], "minibatch")
# model_Ridge = Ridge(k, X_train.shape[1], "minibatch")
model.fit(X_train, Y_train_encoded)
yhat = model.predict(X_test)
model.plot()
print(" ")
print("=========Scratch Classification Report=======")
# Accuracy Report
print("Accuracy Report: ", model.scratch_accuracy(y_test, yhat))

report = model.scratch_classification_report(y_test,yhat)
# Precision Report
print("Precision Report: ", report['precision'])
# Recall Report
print("Recall Report: ", report['recall'])
# F1 Score Report
print("F1 Score Report: ", report['f1_score'])
print(" ")
# weighted report
print("==Weighted Report==")
print("Weighted Precision Report: ", model.weighted_precision(pd.Series(yhat), report['precision']))
print("Weighted Recall Report: ", model.weighted_recall(pd.Series(yhat), report['recall']))
print("Weighted F1_Score Report: ", model.weighted_f1(pd.Series(yhat), report['f1_score']))
print(" ")
# macro report
print("==Macro Report==")
print("Macro Precision Report: ", model.macro_precision(report['precision']))
print("Macro Recall Report: ", model.macro_recall(report['recall']))
print("Macro F1_Score Report: ", model.macro_f1(report['f1_score']))
print(" ")

# Accuracy Report with Sklearn Model 
print("=========Scikit Learn Classification Report=======")
print("Report: ", classification_report(y_test, yhat))
print("support : indicate the total number of samples")
